# Data Mining Project: Milestone 1 (Part 1)
### Web Scraping (Part 1): Scrap movie info and user reviews from rottentomatoes.com 
1. Obtain a list of urls of the latest 250 movies with DVD or Streaming options available  
2. For each movie, scrap the information and ratings. (Structured Data)
3. For each movie, scrap the user reviews. (Unstructured Data)

In [ ]:
# !pip install selenium bs4 joblib

In [4]:
import time                  
import requests
from bs4 import BeautifulSoup
from selenium import webdriver                    
from selenium.webdriver.common.keys import Keys                      
import re
import pandas as pd
import joblib

In [10]:
def find_latest_list_of_movies(URL = 'https://www.rottentomatoes.com/browse/dvd-streaming-all', maxcount = 250):
    # Launch Firefox using selenium package
    browser = webdriver.Firefox(executable_path="./geckodriver-v0.25.0-win64/geckodriver.exe")  
    
    # Browse to the page listing all movies with DVD or Streaming options available  
    browser.get(URL) 
    time.sleep(2) # Wait for "Show More" button to appear 
    
    # Click the "Show More" button to get all the required movies listed in the page.
    total_click_counts = int(maxcount/32)+2
    for j in range(total_click_counts):
        browser.find_element_by_class_name('mb-load-btn').click() # Click "Show More"
        time.sleep(1.5) # Wait for button to appear 
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    # Close the browser
    browser.close() 
    
    # Obtain the url to the page containing the movie information for each movie
    partial_movies_urls = [link.find("a").get("href") for link in soup.find_all("div", class_="movie_info")]
    each_movies_urls = ["https://www.rottentomatoes.com" + j for j in partial_movies_urls]
    
    return each_movies_urls[:maxcount]

def get_movie_info_and_rating(url_movie):
    # For each movie, we will scrap all the relevant information and ratings of the movie
    content = requests.get(url_movie)
    soup = BeautifulSoup(content.text, 'html.parser')

    movie_info = {}

    try:
        # Title
        title = soup.find("h1", class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").text
        
        # Critics Consensus
        critics_consensus = soup.find("p", class_="mop-ratings-wrap__text mop-ratings-wrap__text--concensus")
        critics_consensus = critics_consensus.text if critics_consensus is not None else ""
    except:
        return {}
        
    # Rotten Tomatoes rating
    try:
        tomatometer = [item.text.strip() for item in soup.find_all("span", class_="mop-ratings-wrap__percentage")][0]
    except:
        tomatometer = None
        
    # Audience rating
    try:
        audience_score = [item.text.strip() for item in soup.find_all("span", class_="mop-ratings-wrap__percentage")][1]
    except:
        audience_score = None
    
    # Number of Rotten Tomatoes rating given to the movie
    try: 
        tomatometer_count = soup.find("small", class_="mop-ratings-wrap__text--small").text.strip()
    except:
        tomatometer_count = None
    
     # Number of user rating given to the movie
    try:    
        user_rating_count = soup.find_all("strong", class_="mop-ratings-wrap__text--small")[1].text.strip().split(":")[1]
    except:
        user_rating_count = None
    
    # Return a dictionary containing all these data
    for item in soup.find_all("li", class_ = "meta-row clearfix"):
        key = item.find("div", class_ = "meta-label subtle").text.strip()
        val = item.find("div", class_ = "meta-value").text.strip()
        movie_info[key] = val
    
    movie_info["title"] = title
    movie_info["critics_consensus"] = critics_consensus
    movie_info["tomatometer"] = tomatometer
    movie_info["audience_score"] = audience_score
    movie_info["tomatometer_count"] = tomatometer_count
    movie_info["user_rating_count"] = user_rating_count
    movie_info["url"] = url_movie

    return movie_info

def get_reviews_list(url_movie):
    page_number = 1
    reviews_list = []
    while True:
        # For each page of reviews, scrap all the reviews and append them to the reviews_list.
        url_review = url_movie + "/reviews?page=" + str(page_number)
        content = requests.get(url_review)
        soup = BeautifulSoup(content.text, 'html.parser')
        item_list = soup.find_all("div", class_="row review_table_row")
        if len(item_list) == 0:
            return reviews_list
        
        for item in item_list:
            review_item = item.find("div", class_ = "the_review")
            if review_item is None:
                return reviews_list
            one_review = review_item.text.strip()
            reviews_list.append(one_review)

        page_number = page_number + 1
        
    

### 1. Obtain the list of urls to the latest 250 movies with DVD or Streaming options available  

In [6]:
%%time
movies_urls = find_latest_list_of_movies(maxcount = 5000)
movies_urls[0:10] # preview a few urls

Wall time: 7min 58s


### 2. For each movie, scrap the information and ratings.

In [ ]:
%%time
movie_info = []
for url_movie in movies_urls:
    movie_info.append(get_movie_info_and_rating(url_movie))
    print("Completed:", url_movie)

In [23]:
movie_info_clean = [j for j in movie_info if j is not None]

In [24]:
# df = pd.DataFrame(movie_info)
df = pd.DataFrame(movie_info_clean)
# df.head() # Preview of the data
# df.shape

#### Temporarily store these data in CSV. 
*These data will be stored in the Hive Data Warehouse in the next milestone.*

In [25]:
df.to_csv("movie_info_2.csv", index=False)

### 3. For each movie, scrap the user reviews.
*This will take approximately 20 minutes.*

In [ ]:
%%time
# We will assign the list of user reviews to a dictionary with the url as the key
movie_reviews = {}
for url_movie in movies_urls:
    reviews = get_reviews_list(url_movie)
    movie_reviews[url_movie] = reviews     
    print("Completed:", url_movie)

In [27]:
movie_reviews[movies_urls[10]][0:10] # Preview some user reviews of a movie

['A slow but deep film, with extraordinary performances. [Full review in Spanish]',
 'As long as it takes',
 'An example of a beautifully crafted and photographed, low-key thriller that doubles as a terrific character study about marginalized people.',
 'Lacks necessary tension and stakes when the circumstances tell us there should definitely be some.',
 'Paul and young Danny Murphy are terrific together, with Paul playing a wounded bear growling his lines and Murphy delivering a fully realized performance.',
 "It won't lay a finger on you or look you in the eyes for longer than you're comfortable with. All told, its existence is difficult to justify, but it's here, and it's not bad.",
 '... captures its harsh wintry setting, while sharp performances elevate the otherwise mediocre material, which struggles to build consistent suspense to match its intriguing concept.',
 "It's a premise in search of a plot.",
 'Every frame is filled with significant, illuminating details.',
 "They easil

#### Temporarily save the dictionary as python pickled data (*.pkl format)
*These data will be stored in the Hive Data Warehouse in the next milestone.*

In [28]:
# Temporarily save the dictionary as python pickled data *.pkl
joblib.dump(movie_reviews, filename = "movie_reviews_5000.pkl")

['movie_reviews_5000.pkl']

In [29]:
len(movie_reviews)

4886